In [2]:
%env PGDATABASE=postgres
%env PGHOST=10.82.96.11
%env PGPORT=5432
%env PGUSER=postgres
%env PGPASSWORD=biohackers

env: PGDATABASE=postgres
env: PGHOST=10.82.96.11
env: PGPORT=5432
env: PGUSER=postgres
env: PGPASSWORD=biohackers


In [3]:
!psql -c "\i /home/jupyter/pagila/pagila-schema.sql"
!psql -c "\i /home/jupyter/pagila/pagila-insert-data.sql"

psql:/home/jupyter/pagila/pagila-schema.sql:8: NOTICE:  drop cascades to 47 other objects
DETAIL:  drop cascades to type pagila.mpaa_rating
drop cascades to type pagila.year
drop cascades to function pagila._group_concat(text,text)
drop cascades to function pagila.film_in_stock(integer,integer)
drop cascades to function pagila.film_not_in_stock(integer,integer)
drop cascades to function pagila.get_customer_balance(integer,timestamp with time zone)
drop cascades to function pagila.inventory_held_by_customer(integer)
drop cascades to function pagila.inventory_in_stock(integer)
drop cascades to function pagila.last_day(timestamp with time zone)
drop cascades to function pagila.last_updated()
drop cascades to sequence pagila.customer_customer_id_seq
drop cascades to table pagila.customer
drop cascades to function pagila.rewards_report(integer,numeric)
drop cascades to function pagila.group_concat(text)
drop cascades to sequence pagila.actor_actor_id_seq
drop cascades to table pagila.actor


In [4]:
!psql -c "SET enable_nestloop=0;SELECT 'postgresql' AS dbms,t.table_catalog,t.table_schema,t.table_name,c.column_name,c.ordinal_position,c.data_type,c.character_maximum_length,n.constraint_type,k2.table_schema,k2.table_name,k2.column_name FROM information_schema.tables t NATURAL LEFT JOIN information_schema.columns c LEFT JOIN(information_schema.key_column_usage k NATURAL JOIN information_schema.table_constraints n NATURAL LEFT JOIN information_schema.referential_constraints r)ON c.table_catalog=k.table_catalog AND c.table_schema=k.table_schema AND c.table_name=k.table_name AND c.column_name=k.column_name LEFT JOIN information_schema.key_column_usage k2 ON k.position_in_unique_constraint=k2.ordinal_position AND r.unique_constraint_catalog=k2.constraint_catalog AND r.unique_constraint_schema=k2.constraint_schema AND r.unique_constraint_name=k2.constraint_name WHERE t.TABLE_TYPE='BASE TABLE' AND t.table_schema NOT IN('information_schema','pg_catalog');"

SET
    dbms    | table_catalog | table_schema |  table_name   |     column_name      | ordinal_position |        data_type         | character_maximum_length | constraint_type | table_schema | table_name | column_name  
------------+---------------+--------------+---------------+----------------------+------------------+--------------------------+--------------------------+-----------------+--------------+------------+--------------
 postgresql | postgres      | pagila       | customer      | customer_id          |                1 | integer                  |                          | PRIMARY KEY     |              |            | 
 postgresql | postgres      | pagila       | payment       | payment_id           |                1 | integer                  |                          | PRIMARY KEY     |              |            | 
 postgresql | postgres      | pagila       | actor         | actor_id             |                1 | integer                  |                         

In [5]:
!psql -c "\d pagila.actor"

                                              Table "pagila.actor"
   Column    |           Type           | Collation | Nullable |                    Default                     
-------------+--------------------------+-----------+----------+------------------------------------------------
 actor_id    | integer                  |           | not null | nextval('pagila.actor_actor_id_seq'::regclass)
 first_name  | text                     |           | not null | 
 last_name   | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "actor_pkey" PRIMARY KEY, btree (actor_id)
    "idx_actor_last_name" btree (last_name)
Referenced by:
    TABLE "pagila.film_actor" CONSTRAINT "film_actor_actor_id_fkey" FOREIGN KEY (actor_id) REFERENCES pagila.actor(actor_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.actor FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [6]:
!psql -c "\d pagila.film_actor"

                        Table "pagila.film_actor"
   Column    |           Type           | Collation | Nullable | Default 
-------------+--------------------------+-----------+----------+---------
 actor_id    | smallint                 |           | not null | 
 film_id     | smallint                 |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "film_actor_pkey" PRIMARY KEY, btree (actor_id, film_id)
    "idx_fk_film_id" btree (film_id)
Foreign-key constraints:
    "film_actor_actor_id_fkey" FOREIGN KEY (actor_id) REFERENCES pagila.actor(actor_id) ON UPDATE CASCADE ON DELETE RESTRICT
    "film_actor_film_id_fkey" FOREIGN KEY (film_id) REFERENCES pagila.film(film_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.film_actor FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [7]:
!psql -c "\d pagila.language"

                                               Table "pagila.language"
   Column    |           Type           | Collation | Nullable |                       Default                        
-------------+--------------------------+-----------+----------+------------------------------------------------------
 language_id | integer                  |           | not null | nextval('pagila.language_language_id_seq'::regclass)
 name        | character(20)            |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "language_pkey" PRIMARY KEY, btree (language_id)
Referenced by:
    TABLE "pagila.film" CONSTRAINT "film_language_id_fkey" FOREIGN KEY (language_id) REFERENCES pagila.language(language_id) ON UPDATE CASCADE ON DELETE RESTRICT
    TABLE "pagila.film" CONSTRAINT "film_original_language_id_fkey" FOREIGN KEY (original_language_id) REFERENCES pagila.language(language_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    las

In [8]:
!psql -c "\d pagila.film"

                                                  Table "pagila.film"
        Column        |           Type           | Collation | Nullable |                   Default                    
----------------------+--------------------------+-----------+----------+----------------------------------------------
 film_id              | integer                  |           | not null | nextval('pagila.film_film_id_seq'::regclass)
 title                | text                     |           | not null | 
 description          | text                     |           |          | 
 release_year         | pagila.year              |           |          | 
 language_id          | smallint                 |           | not null | 
 original_language_id | smallint                 |           |          | 
 rental_duration      | smallint                 |           | not null | 3
 rental_rate          | numeric(4,2)             |           | not null | 4.99
 length               | smallint         

In [9]:
!psql -c "\d pagila.film_category"

                      Table "pagila.film_category"
   Column    |           Type           | Collation | Nullable | Default 
-------------+--------------------------+-----------+----------+---------
 film_id     | smallint                 |           | not null | 
 category_id | smallint                 |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "film_category_pkey" PRIMARY KEY, btree (film_id, category_id)
Foreign-key constraints:
    "film_category_category_id_fkey" FOREIGN KEY (category_id) REFERENCES pagila.category(category_id) ON UPDATE CASCADE ON DELETE RESTRICT
    "film_category_film_id_fkey" FOREIGN KEY (film_id) REFERENCES pagila.film(film_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.film_category FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [10]:
!psql -c "\d pagila.inventory"

                                                Table "pagila.inventory"
    Column    |           Type           | Collation | Nullable |                        Default                         
--------------+--------------------------+-----------+----------+--------------------------------------------------------
 inventory_id | integer                  |           | not null | nextval('pagila.inventory_inventory_id_seq'::regclass)
 film_id      | smallint                 |           | not null | 
 store_id     | smallint                 |           | not null | 
 last_update  | timestamp with time zone |           | not null | now()
Indexes:
    "inventory_pkey" PRIMARY KEY, btree (inventory_id)
    "idx_store_id_film_id" btree (store_id, film_id)
Foreign-key constraints:
    "inventory_film_id_fkey" FOREIGN KEY (film_id) REFERENCES pagila.film(film_id) ON UPDATE CASCADE ON DELETE RESTRICT
    "inventory_store_id_fkey" FOREIGN KEY (store_id) REFERENCES pagila.store(store_id) ON UPDA

In [11]:
!psql -c "\d pagila.store"

                                                Table "pagila.store"
      Column      |           Type           | Collation | Nullable |                    Default                     
------------------+--------------------------+-----------+----------+------------------------------------------------
 store_id         | integer                  |           | not null | nextval('pagila.store_store_id_seq'::regclass)
 manager_staff_id | smallint                 |           | not null | 
 address_id       | smallint                 |           | not null | 
 last_update      | timestamp with time zone |           | not null | now()
Indexes:
    "store_pkey" PRIMARY KEY, btree (store_id)
    "idx_unq_manager_staff_id" UNIQUE, btree (manager_staff_id)
Foreign-key constraints:
    "store_address_id_fkey" FOREIGN KEY (address_id) REFERENCES pagila.address(address_id) ON UPDATE CASCADE ON DELETE RESTRICT
Referenced by:
    TABLE "pagila.customer" CONSTRAINT "customer_store_id_fkey" FOREIGN 

Outer join the film table to the film_category table by film_id. Then join the film_category table to category by category_id. Use where to keep only columns where the name of the category is horror. Order the rows by reverse alphabetical order using the desc command and then limit to display the first 10 rows. 

In [12]:
!psql -c "\d pagila.category"

                                               Table "pagila.category"
   Column    |           Type           | Collation | Nullable |                       Default                        
-------------+--------------------------+-----------+----------+------------------------------------------------------
 category_id | integer                  |           | not null | nextval('pagila.category_category_id_seq'::regclass)
 name        | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "category_pkey" PRIMARY KEY, btree (category_id)
Referenced by:
    TABLE "pagila.film_category" CONSTRAINT "film_category_category_id_fkey" FOREIGN KEY (category_id) REFERENCES pagila.category(category_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.category FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [13]:
!psql -c "\d pagila.customer"

                                                Table "pagila.customer"
   Column    |           Type           | Collation | Nullable |                        Default                        
-------------+--------------------------+-----------+----------+-------------------------------------------------------
 customer_id | integer                  |           | not null | nextval('pagila.customer_customer_id_seq1'::regclass)
 store_id    | smallint                 |           | not null | 
 first_name  | text                     |           | not null | 
 last_name   | text                     |           | not null | 
 email       | text                     |           |          | 
 address_id  | smallint                 |           | not null | 
 activebool  | boolean                  |           | not null | true
 create_date | date                     |           | not null | 'now'::text::date
 last_update | timestamp with time zone |           |          | now()
 active      | 

In [14]:
!psql -c "\d pagila.city"

                                             Table "pagila.city"
   Column    |           Type           | Collation | Nullable |                   Default                    
-------------+--------------------------+-----------+----------+----------------------------------------------
 city_id     | integer                  |           | not null | nextval('pagila.city_city_id_seq'::regclass)
 city        | text                     |           | not null | 
 country_id  | smallint                 |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "city_pkey" PRIMARY KEY, btree (city_id)
    "idx_fk_country_id" btree (country_id)
Foreign-key constraints:
    "city_country_id_fkey" FOREIGN KEY (country_id) REFERENCES pagila.country(country_id) ON UPDATE CASCADE ON DELETE RESTRICT
Referenced by:
    TABLE "pagila.address" CONSTRAINT "address_city_id_fkey" FOREIGN KEY (city_id) REFERENCES pagila.city(city_id) ON UPDATE CASCADE ON 

In [15]:
!psql -c "\d pagila.address"

                                               Table "pagila.address"
   Column    |           Type           | Collation | Nullable |                      Default                       
-------------+--------------------------+-----------+----------+----------------------------------------------------
 address_id  | integer                  |           | not null | nextval('pagila.address_address_id_seq'::regclass)
 address     | text                     |           | not null | 
 address2    | text                     |           |          | 
 district    | text                     |           | not null | 
 city_id     | smallint                 |           | not null | 
 postal_code | text                     |           |          | 
 phone       | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "address_pkey" PRIMARY KEY, btree (address_id)
    "idx_fk_city_id" btree (city_id)
Foreign-key cons

In [16]:
!psql -c "\d pagila.staff"

                                              Table "pagila.staff"
   Column    |           Type           | Collation | Nullable |                    Default                     
-------------+--------------------------+-----------+----------+------------------------------------------------
 staff_id    | integer                  |           | not null | nextval('pagila.staff_staff_id_seq'::regclass)
 first_name  | text                     |           | not null | 
 last_name   | text                     |           | not null | 
 address_id  | smallint                 |           | not null | 
 email       | text                     |           |          | 
 store_id    | smallint                 |           | not null | 
 active      | boolean                  |           | not null | true
 username    | text                     |           | not null | 
 password    | text                     |           |          | 
 last_update | timestamp with time zone |           | not null 

In [17]:
!psql -c "\d pagila.country"

                                               Table "pagila.country"
   Column    |           Type           | Collation | Nullable |                      Default                       
-------------+--------------------------+-----------+----------+----------------------------------------------------
 country_id  | integer                  |           | not null | nextval('pagila.country_country_id_seq'::regclass)
 country     | text                     |           | not null | 
 last_update | timestamp with time zone |           | not null | now()
Indexes:
    "country_pkey" PRIMARY KEY, btree (country_id)
Referenced by:
    TABLE "pagila.city" CONSTRAINT "city_country_id_fkey" FOREIGN KEY (country_id) REFERENCES pagila.country(country_id) ON UPDATE CASCADE ON DELETE RESTRICT
Triggers:
    last_updated BEFORE UPDATE ON pagila.country FOR EACH ROW EXECUTE FUNCTION pagila.last_updated()



In [18]:
!psql -c "\d pagila.rental"

                                               Table "pagila.rental"
    Column    |           Type           | Collation | Nullable |                     Default                      
--------------+--------------------------+-----------+----------+--------------------------------------------------
 rental_id    | integer                  |           | not null | nextval('pagila.rental_rental_id_seq'::regclass)
 rental_date  | timestamp with time zone |           | not null | 
 inventory_id | integer                  |           | not null | 
 customer_id  | smallint                 |           | not null | 
 return_date  | timestamp with time zone |           |          | 
 staff_id     | smallint                 |           | not null | 
 last_update  | timestamp with time zone |           | not null | now()
Indexes:
    "rental_pkey" PRIMARY KEY, btree (rental_id)
    "idx_fk_inventory_id" btree (inventory_id)
    "idx_unq_rental_rental_date_inventory_id_customer_id" UNIQUE, btree 

In [19]:
!psql -c "\d pagila.payment"

                                               Table "pagila.payment"
    Column    |           Type           | Collation | Nullable |                      Default                       
--------------+--------------------------+-----------+----------+----------------------------------------------------
 payment_id   | integer                  |           | not null | nextval('pagila.payment_payment_id_seq'::regclass)
 customer_id  | smallint                 |           | not null | 
 staff_id     | smallint                 |           | not null | 
 rental_id    | integer                  |           | not null | 
 amount       | numeric(5,2)             |           | not null | 
 payment_date | timestamp with time zone |           | not null | 
Indexes:
    "payment_pkey" PRIMARY KEY, btree (payment_id)
    "idx_fk_customer_id" btree (customer_id)
    "idx_fk_staff_id" btree (staff_id)
Foreign-key constraints:
    "customer_id_fkey" FOREIGN KEY (customer_id) REFERENCES pagila.custom

Inner join the inventory table to the rental table on inventory_id and display the store_id, inventory_id, and film_id. Only include the rows where store_id is 2 and sort by film_id in ascending order.

In [22]:
!psql -c "select i.store_id, i.inventory_id, i.film_id from pagila.inventory i inner join pagila.rental on i.inventory_id = pagila.rental.inventory_id where store_id = 2 order by film_id limit 10"

 store_id | inventory_id | film_id 
----------+--------------+---------
        2 |            6 |       1
        2 |            6 |       1
        2 |            6 |       1
        2 |            6 |       1
        2 |            6 |       1
        2 |            7 |       1
        2 |            7 |       1
        2 |            7 |       1
        2 |            7 |       1
        2 |            8 |       1
(10 rows)



Inner join the store table to the customer table on store_id and display the store_id, first_name, and address_id, only including the rows where address_id > 10 and sorting by first_name.

In [26]:
!psql -c "select c.store_id, c.first_name, c.address_id from pagila.customer c inner join pagila.store s on c.store_id = s.store_id where c.address_id > 10 order by c.first_name limit 10"

 store_id | first_name | address_id 
----------+------------+------------
        2 | AARON      |        380
        1 | ADAM       |        372
        2 | ADRIAN     |        531
        2 | AGNES      |        221
        1 | ALAN       |        394
        1 | ALBERT     |        357
        2 | ALBERTO    |        574
        2 | ALEX       |        459
        2 | ALEXANDER  |        444
        2 | ALFRED     |        428
(10 rows)



Inner join the city table to the country table on country_id and display the city, country_id, and country. Only include cities in Brazil and sort the cities in reverse alphabetical order.

In [39]:
!psql -c "select city.city, city.country_id, country.country from pagila.city city inner join pagila.country country on city.country_id = country.country_id where country.country = 'Brazil' order by city desc limit 10"

         city         | country_id | country 
----------------------+------------+---------
 Vitria de Santo Anto |         15 | Brazil
 Vila Velha           |         15 | Brazil
 Sorocaba             |         15 | Brazil
 So Leopoldo          |         15 | Brazil
 So Bernardo do Campo |         15 | Brazil
 Santo Andr           |         15 | Brazil
 Santa Brbara dOeste  |         15 | Brazil
 Rio Claro            |         15 | Brazil
 Poos de Caldas       |         15 | Brazil
 Po                   |         15 | Brazil
(10 rows)



Outer join the film_category table to the film table on film_id and category_id and show the columns film_id, title, and name. Only include results where name is 'Horror' and order by title in reverse alphabetical order.

In [ ]:
!psql -c "select pagila.film.film_id, pagila.film.title, pagila.category.name from pagila.film FULL OUTER JOIN pagila.film_category on pagila.film.film_id = pagila.film_category.film_id FULL OUTER JOIN pagila.category on pagila.film_category.category_id = pagila.category.category_id WHERE pagila.category.name = 'Horror' ORDER BY pagila.film.title DESC LIMIT 10

 film_id |         title         |  name  
---------+-----------------------+--------
     998 | ZHIVAGO CORE          | Horror
     995 | YENTL IDAHO           | Horror
     990 | WORLD LEATHERNECKS    | Horror
     965 | WATERSHIP FRONTIER    | Horror
     922 | UNDEFEATED DALMATIONS | Horror
     909 | TREASURE COMMAND      | Horror
     904 | TRAIN BUNCH           | Horror
     885 | TEXAS WATCH           | Horror
     881 | TEMPLE ATTRACTION     | Horror
     876 | TARZAN VIDEOTAPE      | Horror
(10 rows)



Outer join the store table to the customer table by store_id. Count the number of occurances of first name per store per name to see if there are any repeat customers. Display the store_id, first_name, and visits with visits in ascending order where store_id is 2. Overall, displaying the most frequent customers to store_id 2 limited to the first 10 rows.

In [3]:
!psql -c "select pagila.store.store_id, pagila.customer.first_name, count(pagila.customer.first_name) as visits from pagila.store FULL OUTER JOIN pagila.customer on pagila.store.store_id = pagila.customer.store_id WHERE pagila.store.store_id = 2 GROUP BY pagila.store.store_id, pagila.customer.first_name ORDER BY visits ASC LIMIT 10"

 store_id | first_name | visits 
----------+------------+--------
        2 | MICHELE    |      1
        2 | SHIRLEY    |      1
        2 | ALLEN      |      1
        2 | EILEEN     |      1
        2 | BONNIE     |      1
        2 | RODNEY     |      1
        2 | RICK       |      1
        2 | DANIEL     |      1
        2 | KYLE       |      1
        2 | RANDALL    |      1
(10 rows)



Outer join the customer table to the payment table by customer_id. Filter the data where payment_id <16060 to get the first 10 orders. Display the first_name of customer, payment_id, and amount. Order the data by payment amount from greatest to lowest with desc.

In [4]:
!psql -c "select pagila.customer.first_name, pagila.payment.payment_id, pagila.payment.amount from pagila.customer FULL OUTER JOIN pagila.payment on pagila.customer.customer_id = pagila.payment.customer_id WHERE pagila.payment.payment_id < 16060 ORDER BY pagila.payment.amount DESC"

 first_name | payment_id | amount 
------------+------------+--------
 PENNY      |      16058 |   8.99
 CASSANDRA  |      16052 |   6.99
 LEAH       |      16057 |   4.99
 CASSANDRA  |      16054 |   4.99
 CASSANDRA  |      16055 |   2.99
 CASSANDRA  |      16050 |   1.99
 LEAH       |      16056 |   1.99
 CASSANDRA  |      16053 |   0.99
 CASSANDRA  |      16051 |   0.99
 KAY        |      16059 |   0.99
(10 rows)



Inner join the staff table with itself on password equals itself where the staff id is not the same. This returns staff members in the dataset that are different people but have the same account password. Use select to display the username and password of the staff with matching passwords. 

In [5]:
!psql -c "select s1.username, s1.password from pagila.staff s1 JOIN pagila.staff s2 on s1.password = s2.password WHERE s1.staff_id !=  s2.staff_id"

 username |                 password                 
----------+------------------------------------------
 Mike     | 8cb2237d0679ca88db6464eac60da96345513964
 Jon      | 8cb2237d0679ca88db6464eac60da96345513964
(2 rows)

